In [ ]:
# 필요한 패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import pandas as pd
import requests
import pickle
from selenium.common.exceptions import WebDriverException

In [ ]:
import pickle
filePath = 'C:/Users/g20/Downloads/지표코드리스트.txt' #미래에셋 데이터 중 코스피에 상장된 종목만 리스트화
with open(filePath,'rb') as lf:
      kospi_total_code = pickle.load(lf)

In [ ]:
kospi_total_code = kospi_total_code.tolist()

In [ ]:
len(kospi_total_code)

791

### Financial Summary 연간 table

In [ ]:
def EV_EBITDA_annual(kospi_code):
    # Web서버 오류등 크롤링 불가능 할 경우 nondata에 담아 목록 출력
    nondata = []

    for code in tqdm(kospi_code):
        try:
            driver = webdriver.Chrome()
            driver.implicitly_wait(15) # 페이지가 다 뜰때까지 최대 10


            driver.get("https://finance.daum.net/quotes/{}#analysis/main".format(code))
            time.sleep(3)
            driver.find_element(By.XPATH,'//*[@id="boxTabs"]/td[7]/a').click()
            time.sleep(3)

            # error발생 크롤링 
            driver.switch_to.frame(driver.find_element(By.XPATH, '''//*[@id="boxContents"]/div[7]/div[1]/iframe'''))
            time.sleep(1)

            driver.switch_to.frame(driver.find_element(By.XPATH, '''//*[@id="frmFS1"]'''))
            time.sleep(1)

            source = driver.page_source # 현재 페이지 크롤링
            html = BeautifulSoup(source)

            # Financial Summary table(연간) 추출
            table = html.find_all('table',class_ = 'tbl')[0]
            table = pd.read_html(str(table))[0]
            table = pd.DataFrame(table) # 반드시 데이터 프레임화 해야한다, 아니면 튜플로 됨 ->지금도 튜플로나오긴함..ㅋㅅㅋ..
            table = table.dropna(how = 'all')
            
            #필요한 EV/EBITDA 지표만 추출
            table = table[['FY-end','EV/EBITDA (배)']]
            table = table.rename( columns ={'EV/EBITDA (배)' : code})

            if code == kospi_code[0]:
                # 재수없게 2019부터 시작 못하는 경우 방지
#                 table_total_annual = pd.DataFrame({'FY-end': ['2019(A)', '2020(A)', '2021(A)', '2022(E)', '2023(E)'],
#                                                    'temp_nm':['2019(A)', '2020(A)', '2021(A)', '2022(E)', '2023(E)']})식으로 df자리 할당
                table_total_annual = pd.DataFrame({'FY-end': table['FY-end'].values.tolist(),
                                                   'temp_nm':table['FY-end'].values.tolist()})
                # 할당된 자리에 첫번째 값 merge  
                table_total_annual = table_total_annual.merge(table, on = 'FY-end', how = 'left')
                table_total_annual.drop(['temp_nm'], axis = 1,inplace = True) #dummy값 삭제
                


            else:
                # FY-end를 기준으로 각 코드별 EV/EBITDA코드 병합
                table_total_annual = table_total_annual.merge(table,on='FY-end',how = 'left') 

        # 다음 금융>기업공시 접근 불가 에러처리
        except WebDriverException:
            nondata.append(code)
            time.sleep(10*6)

        driver.quit() #한번 크롤링 종료시 창닫기
        time.sleep(3)
    if len(nondata) == 0:
        return table_total_annual

    else:
        return print('{}는 EV_EBITDA가 없습니다.'.format(nondata)),table_total_annual

In [ ]:
temp = EV_EBITDA_annual(kospi_total_code)

In [ ]:
dump = temp[1] # 왜 연간은 tuple 처리가 될까??????
display(dump)

In [ ]:
# 웹크롤링 시간이 오래 걸려 data dump 
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBITDA_annual_kospi.csv'
with open(filepath,'wb') as lf:
    pickle.dump(dump, lf)

In [ ]:
# anuual 잘 되었는지 출력 확인
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBITDA_annual_kospi.csv'
with open(filepath,'rb') as lf:
    data = pickle.load( lf)
    display(data)

,FY-end,A095570,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A298050,A298020,A298000,A093370,A081660,A005870,A079980,A005010,A000540,A003280
0,2019(A),6.5,4.0,6.7,5.8,15.6,6.9,4.2,7.0,8.6,...,6.7,4.6,5.6,22.3,7.2,6.6,11.0,NaN,6.2,31.2
1,2020(A),5.7,30.1,7.7,4.8,18.9,6.8,4.0,NaN,8.2,...,10.6,4.5,8.9,33.9,7.2,1.9,4.0,4.3,10.8,31.3
3,2021(A),4.9,26.5,10.1,4.8,16.7,7.8,3.6,46.9,6.6,...,6.8,2.0,8.7,22.5,3.9,5.1,8.8,2.5,5.2,72.7
4,2022(E),4.8,NaN,6.6,4.8,NaN,NaN,3.4,13.8,5.8,...,5.2,2.2,7.9,6.8,3.0,NaN,NaN,NaN,NaN,NaN
5,2023(E),5.7,NaN,7.9,4.7,NaN,NaN,3.2,11.0,4.9,...,4.7,1.6,4.2,5.7,2.4,NaN,NaN,NaN,NaN,NaN


### Financial Summary 분기 table
### 현재 값이 추가되어 다시 돌려야할 것 같음

In [ ]:
def EV_EBITDA_half(kospi_code):

    nondata = []

    for code in tqdm(kospi_code):
        try:
            driver = webdriver.Chrome()
            driver.implicitly_wait(15) # 페이지가 다 뜰때까지 최대 10

            driver.get("https://finance.daum.net/quotes/{}#analysis/main".format(code))
            time.sleep(3)

            driver.find_element(By.XPATH,'//*[@id="boxTabs"]/td[7]/a').click()
            time.sleep(3)

            # Finacial Summary 분기로 접근 
            driver.switch_to.default_content()
            driver.switch_to.frame(driver.find_element(By.XPATH, '''//*[@id="boxContents"]/div[7]/div[1]/iframe'''))
            time.sleep(1)

            # 분기 click
            driver.find_element(By.XPATH, '''//*[@id="cns_Tab2"]/a''').click()


            # table접근을 위한 접근 처리
            driver.switch_to.frame(driver.find_element(By.XPATH, '''//*[@id="frmFS1"]'''))
            time.sleep(1)

            source = driver.page_source # 현재 페이지 크롤링
            html = BeautifulSoup(source)

            # Financial Summary table(연간) 추출
            table = html.find_all('table',class_ = 'tbl')[0]
            table = pd.read_html(str(table))[0]
            table = pd.DataFrame(table)

            table = table.dropna(how = 'all') # 모두 nan행 제거 
            table = table[['FY-end','EV/EBITDA (배)']]
            table = table.rename( columns ={'EV/EBITDA (배)': code})


            if code == kospi_code[0]:
                # 재수없게 처음부터(ex2021.09(A)부터 상장이되지 않아) 시작 못하는 경우 방지(df자리 할당)
                
#                 table_total_half = pd.DataFrame({'FY-end':['2021.09(A)', '2021.12(A)', '2022.03(A)', '2022.06(E)', '2022.09(E)'],
#                                                 'temp_nm':['2021.09(A)', '2021.12(A)', '2022.03(A)', '2022.06(E)', '2022.09(E)']})식으로 df자리 할당
                table_total_half = pd.DataFrame({'FY-end': table['FY-end'].values.tolist(),
                                                 'temp_nm':table['FY-end'].values.tolist()})
                # 할당된 자리에 첫번째 값 merge    
                table_total_half = table_total_half.merge(table,on='FY-end',how = 'left')
                table_total_half.drop(['temp_nm'],axis=1, inplace = True)
                
            else:
                # FY-end를 기준으로 각 코드별 EV/EBITDA코드 병합 
                table_total_half = table_total_half.merge(table, on = 'FY-end', how = 'left')
        
        # 다음 금융>기업공시 접근 불가 에러처리
        except WebDriverException:
            nondata.append(nondata)
            time.sleep(10*6)
        
        driver.quit()
        time.sleep(3)
        display(table_total_half)
        
    if len(nondata) == 0:
        return table_total_half

    else:
        return print('{}는 EV_EBITDA가 없습니다.'.format(nondata)), table_total_half

In [ ]:
temp = EV_EBITDA_half(kospi_total_code)
display(temp)

In [ ]:
# 웹크롤링 시간이 오래 걸려 data dump 
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBIDTA_half.csv'
with open(filepath,'wb') as lf:
    pickle.dump(temp,lf)

In [ ]:
# half 잘 되었는지 출력 확인
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBIDTA_half.csv'
with open(filepath, 'rb') as lf:
    data2 = pickle.load(lf)
display(data2)

,FY-end,A095570,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A298050,A298020,A298000,A093370,A081660,A005870,A079980,A005010,A000540,A003280
0,2021.09(A),18.5,NaN,20.9,15.1,54.2,31.1,14.6,3679.3,56.5,...,NaN,NaN,NaN,96.1,NaN,12.3,47.2,NaN,11.5,NaN
1,2021.12(A),21.0,120.6,53.2,18.9,159.2,23.4,15.7,108.8,9.3,...,29.2,8.1,76.0,31.4,46.2,27.0,398.9,5.1,46.3,107.7
2,2022.03(A),18.4,131.6,36.5,19.0,54.0,30.2,14.7,172.5,75.7,...,26.1,13.7,79.4,37.8,11.6,NaN,NaN,6.7,8.6,152.7
3,2022.06(E),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022.09(E),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***

## #'A064350'에 관한 연간 지표가 누락됨.
이에 관해 data_csv에 추가로 나중에 merge함 ->왜 누락됐을까,..???

In [ ]:
#분기는 제대로 잘 출력됨(791개값 출력)
#연간이 'A064350'이 누락되어 이 코드를 찾아내고 
set(data2.columns.values.tolist())-set(data.columns.values.tolist())

{'A064350'}

In [ ]:
temp = EV_EBITDA_annual(['A064350']) #위의 set을 복사붙여넣기해서 다시 값 추출
data = data.merge(temp,on='FY-end',how = 'left')  #연간 csv파일로 dump시킨 값을 누락값('A064350')을 값이 병합

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.23s/it]


In [ ]:
data

,FY-end,A095570,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A298020,A298000,A093370,A081660,A005870,A079980,A005010,A000540,A003280,A064350
0,2019(A),6.5,4.0,6.7,5.8,15.6,6.9,4.2,7.0,8.6,...,4.6,5.6,22.3,7.2,6.6,11.0,NaN,6.2,31.2,NaN
1,2020(A),5.7,30.1,7.7,4.8,18.9,6.8,4.0,NaN,8.2,...,4.5,8.9,33.9,7.2,1.9,4.0,4.3,10.8,31.3,19.8
2,2021(A),4.9,26.5,10.1,4.8,16.7,7.8,3.6,46.9,6.6,...,2.0,8.7,22.5,3.9,5.1,8.8,2.5,5.2,72.7,27.6
3,2022(E),4.8,NaN,6.6,4.8,NaN,NaN,3.4,13.8,5.8,...,2.2,7.9,6.8,3.0,NaN,NaN,NaN,NaN,NaN,22.6
4,2023(E),5.7,NaN,7.9,4.7,NaN,NaN,3.2,11.0,4.9,...,1.6,4.2,5.7,2.4,NaN,NaN,NaN,NaN,NaN,15.1


In [ ]:
import pickle

In [ ]:
# 웹크롤링 시간이 오래 걸려 data dump 
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBITDA_annual_kospi_final.csv'
with open(filepath,'wb') as lf:
    pickle.dump(data, lf)

In [ ]:
filepath = 'C:/Users/g20/OneDrive/바탕 화면/EV_EBITDA/EV_EBITDA_annual_kospi_final.csv'
with open(filepath,'rb') as lf:
    data = pickle.load(lf)

In [ ]:
data

,FY-end,A095570,A006840,A027410,A282330,A138930,A001460,A001040,A079160,A000120,...,A298020,A298000,A093370,A081660,A005870,A079980,A005010,A000540,A003280,A064350
0,2019(A),6.5,4.0,6.7,5.8,15.6,6.9,4.2,7.0,8.6,...,4.6,5.6,22.3,7.2,6.6,11.0,NaN,6.2,31.2,NaN
1,2020(A),5.7,30.1,7.7,4.8,18.9,6.8,4.0,NaN,8.2,...,4.5,8.9,33.9,7.2,1.9,4.0,4.3,10.8,31.3,19.8
2,2021(A),4.9,26.5,10.1,4.8,16.7,7.8,3.6,46.9,6.6,...,2.0,8.7,22.5,3.9,5.1,8.8,2.5,5.2,72.7,27.6
3,2022(E),4.8,NaN,6.6,4.8,NaN,NaN,3.4,13.8,5.8,...,2.2,7.9,6.8,3.0,NaN,NaN,NaN,NaN,NaN,22.6
4,2023(E),5.7,NaN,7.9,4.7,NaN,NaN,3.2,11.0,4.9,...,1.6,4.2,5.7,2.4,NaN,NaN,NaN,NaN,NaN,15.1


***

### 예린언니와 지표 합치기 위한 전처리 

In [ ]:
# 지표 합치기 전 전처리 
def preproc_ev_ebi_indic(data): #ev/ebitda 데이터프레임 input
    #df의 stock_code변경( ex A00530->00530)
    stock_code = []
    col_list = data.columns.values.tolist()
    for i in range(1,len(col_list)):
        re_code = col_list[i][1:]
        stock_code.append(re_code)
    
    #[stock_code, 2019(A),2020(A),2021(A),2022(E),2023(E)]으로 출력//이때, stock_code는 A00530꼴
    temp = data.T
    col = temp.iloc[0].values.tolist()
    col.insert(0,'stock_code')

    temp = temp.reset_index()
    temp.columns = col
    temp = temp.iloc[1:]
    
    # A00530->00530 변경 적용
    temp['stock_code'] = stock_code
    
    return temp

In [ ]:
preproc_ev_ebi_indic(data) #791개에 대한 연간 값이 제대로 추출됨.

,stock_code,2019(A),2020(A),2021(A),2022(E),2023(E)
1,095570,6.5,5.7,4.9,4.8,5.7
2,006840,4.0,30.1,26.5,NaN,NaN
3,027410,6.7,7.7,10.1,6.6,7.9
4,282330,5.8,4.8,4.8,4.8,4.7
5,138930,15.6,18.9,16.7,NaN,NaN
...,...,...,...,...,...,...
787,079980,11.0,4.0,8.8,NaN,NaN
788,005010,NaN,4.3,2.5,NaN,NaN
789,000540,6.2,10.8,5.2,NaN,NaN
790,003280,31.2,31.3,72.7,NaN,NaN


In [ ]:
preproc_ev_ebi_indic(data2) #791개에 대한 분기값이 제대로 추출됨.

,stock_code,2021.09(A),2021.12(A),2022.03(A),2022.06(E),2022.09(E)
1,095570,18.5,21.0,18.4,NaN,NaN
2,006840,NaN,120.6,131.6,NaN,NaN
3,027410,20.9,53.2,36.5,NaN,NaN
4,282330,15.1,18.9,19.0,NaN,NaN
5,138930,54.2,159.2,54.0,NaN,NaN
...,...,...,...,...,...,...
787,005870,12.3,27.0,NaN,NaN,NaN
788,079980,47.2,398.9,NaN,NaN,NaN
789,005010,NaN,5.1,6.7,NaN,NaN
790,000540,11.5,46.3,8.6,NaN,NaN
